In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly import __version__
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot 
init_notebook_mode(connected = True)
cf.go_offline()
import plotly.express as px

In [2]:
def basic_info(df, *isos):  #this produces a total deaths graph and a new cases graph for the countries you pass in for the past month
    for iso in isos:
        
        #TO BE USED BY BOTH GRAPHS
        
        
        sns.set_style('whitegrid')  #dark background
        country_name = df[df['iso_code']==iso]['location'].iloc[0]  #this brings the df where the iso code is the one u have entered and then brings the location associated with it. 
         #.iloc[0] just brings the location name rather than a list of the location name repeated 1000 times
      
        df_country = df[df['iso_code']==iso].tail(30) #this builds the df we are going to use. its the data from the country entered from the past 7 days.
        dates = df_country['date']   #the list of dates that will be plotted on x axis

        
        #THE NEW CASES GRAPH
        
        new_cases = df_country['new_cases']
      
        plt.figure(figsize =(10,5))  
      
        fig1 = sns.lineplot('date','new_cases', data = df_country, color = 'red', marker = 'o', markersize = 10)  #plots the new cases against the date (from past week). uses data from df_country
      
        fig1.set_xticks(np.arange(1,31,5))
         
        plt.xlim(xmin = dates.iloc[0])  #first date is  at origin
        plt.ylim(ymin = (new_cases.min()) * 0.95, ymax = (new_cases.max()  * 1.05))  #the min and max are multiplied by 0.9 and 1.1 respectively so the graph has leeway - > no points ON axes
      
        plt.title(f'{country_name} New Cases: Last 30 days') #title of graph uses country name, defined above
      
        plt.xlabel('Date')
        plt.ylabel('New Cases')
            
    
    
        #THE TOTAL DEATHS GRAPH
        
        total_deaths = df_country['total_deaths']
      
        plt.figure(figsize = (10,5))  #new figure, same size
        plt.title(f'{country_name} Total Death Count')
      
        fig2 = sns.lineplot('date', 'total_deaths', data = df_country, marker = 'v', markersize = 10, color = 'blue' )
        fig2.set_xticks(np.arange(1,31,5))
      
        plt.xlim(xmin = dates.iloc[0])
        plt.ylim(ymin = total_deaths.min() * 0.999 , ymax = total_deaths.max() * 1.001)
    
      
        plt.xlabel('Date')
        plt.ylabel('Total Deaths')
      
      
      
        plt.tight_layout() #neatens layout


In [3]:
def country_compare_death_rate(df, 
                               iso1, iso2, days = 30): #3rd param is optional- can choose timeline of graph
    
    #CREATING A DATAFRAME WHERE IT CONTAINS DATA FROM BOTH COUNTRIES OVER THE PAST 7 DAYS

    df_iso1 = df[df['iso_code']==iso1].tail(days)
    country_1 = df_iso1['location'].iloc[0]   #just grabbing the location name
    
    df_iso2 = df[df['iso_code']==iso2].tail(days)
    country_2 = df_iso2['location'].iloc[0]   
    

    df_both = df_iso1.append(df_iso2)
    
    
    
    #PLOTTING GRAPH OF NEW DEATHS OF BOTH COUNTRIES
    plt.figure(figsize= (10,5))
    sns.set_style('whitegrid')
    
 
    sns.lineplot('date','new_deaths', data = df_both, hue = 'location')

    plt.xticks(np.arange(0,31,4))
    
    plt.xlim(xmin = df_both['date'].iloc[0])  #first date at #origin
    plt.ylim(ymin = 0)  #y axis starts from 0
    plt.xlabel('Date')
    plt.ylabel('New Deaths')
    plt.title(f'Death rate (past {days} days)')
    plt.legend([f'{country_1}',f'{country_2}'], loc = 'best', fontsize = 'large', shadow = True)
    
    

In [4]:
def country_compare_case_rate(df,iso1, iso2, days = 30): #3rd param is optional- can choose timeline of graph
    df_iso1 = df[df['iso_code']==iso1].tail(days)
    country_1 = df_iso1['location'].iloc[0]   #just grabbing the location name
    
    df_iso2 = df[df['iso_code']==iso2].tail(days)
    country_2 = df_iso2['location'].iloc[0]   

    df_both = df_iso1.append(df_iso2)
    plt.figure(figsize= (10,5))
    sns.set_style('whitegrid')
    
 
    sns.lineplot('date','new_cases', data = df_both, hue = 'location')

    plt.xticks(np.arange(0,31,4))    
    plt.xlim(xmin = df_both['date'].iloc[0])  #first date at #origin
    plt.ylim(ymin = 0)  #y axis starts from 0
    plt.xlabel('Date')
    plt.ylabel('New Cases')
    plt.title(f'Case rate (past {days} days)')
    plt.legend([f'{country_1}',f'{country_2}'], loc = 'best', fontsize = 'large', shadow = True)


In [5]:
def icountry_compare(df, iso1, iso2, days = 30):  #again, 3rd param is optional - timeline choice
    
    country_1 = df[df['iso_code']==iso1]['location'].iloc[0]
    df_iso1 = df[df['iso_code']==iso1].tail(days)[['date','new_deaths']] #df w date and new deaths
 
    df_iso2 = df[df['iso_code']==iso2].tail(days)[['date','new_deaths']]
    country_2 = df[df['iso_code']==iso2]['location'].iloc[0]

    df_iso1new = df_iso1.set_index('date')
    df_iso2new = df_iso2.set_index('date') #dataframes where date is index. makes it possible to concat them

    df_both = pd.concat([df_iso1new, df_iso2new], axis = 1) #concatenating dfs
    df_both.columns  = [f'{country_1}',f'{country_2}']
    
    
    
    df_both.iplot(title = f'New Deaths per day- {country_1}, {country_2} (past {days} days)', color = ['orange','navy'])
  
  


In [10]:
def bubbleScatter(df, x_param, y_param, size_param, hover_name_param, x_axis_type, y_axis_type,
                  title_p, color_param = 'red'):
    fig = go.Figure()
    
    if (color_param == 'red'):
        fig = px.scatter(df, x = x_param, y = y_param, size = size_param, trendline = 'ols', 
                     hover_name = hover_name_param, color_discrete_sequence=['red'])
    else:
        fig = px.scatter(df, x = x_param, y = y_param, size = size_param, trendline = 'ols', 
                     hover_name = hover_name_param, color = color_param)
    
    fig.update_layout(xaxis_type = x_axis_type, yaxis_type = y_axis_type)
    fig.update_layout(
        width = 1200,
        height = 650,
        title = title_p
        )
    fig.show()

In [ ]:
def regPlotter(df, x_param, y_param, scale, title):
    fig = plt.figure(figsize = (15,15))
    ax = sns.regplot(x = x_param, y = y_param, data = df)

    ax.set_yscale(scale)
    plt.title(title)
    plt.show()

In [ ]:
def sns_barplot(x_param, y_param, df, title):
    fig = plt.figure(figsize = (15, 8))
    sns.barplot(x = x_param, y= y_param,data = df) 
    plt.title(title)

In [ ]:
def i_barplot(x_p, y_p, color_p, barmode_p, df):
    fig = px.bar(df, x=x_p, y=y_p,
             color=color_p, barmode = barmode_p,
             height=700)
    fig.show()

In [7]:
def boxPlotter(x_p, y_p, df, orient_p, title):
    fig = plt.figure(figsize = (20,17))
    ax = sns.boxplot(y = y_p, x = x_p, data = df, orient = orient_p)
    plt.title(title, fontsize =20)